## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])


In [3]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()
unique_values

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_type_counts
# Define the threshold for "Other" (example: less than 500)
threshold = 100

# Replace APPLICATION_TYPE values that are below the threshold with "Other"
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].apply(lambda x: "Other" if application_type_counts[x] < threshold else x)

# Verify the changes
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# Choose a cutoff value
cutoff = 100

# Create a list of application types to be replaced
application_types_to_replace = application_type_counts[application_type_counts < cutoff].index.tolist()

# Display the list of application types to be replaced
print(application_types_to_replace)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

['T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']


APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Define the cutoff value for "Other" 
cutoff = 100

# Create a list of classifications to be replaced
classifications_to_replace = classification_counts[classification_counts < cutoff].index.tolist()

# Replace the classifications in the dataframe with 'Other'
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(classifications_to_replace, 'Other')

# Verify the changes
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
# Get the value counts for CLASSIFICATION
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Display classifications with counts greater than 1
classification_counts_gt1 = classification_counts[classification_counts > 1]

# Display the value counts
print(classification_counts_gt1)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Define the cutoff value for "Other"
cutoff = 100

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_encoded = pd.get_dummies(application_df)

# Display the first few rows of the encoded DataFrame
print(application_df_encoded.head())

   STATUS  ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0       1     5000              1                   False   
1       1   108590              1                   False   
2       1     5000              0                   False   
3       1     6692              1                   False   
4       1   142590              1                   False   

   APPLICATION_TYPE_T10  APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  \
0                  True                 False                False   
1                 False                 False                 True   
2                 False                 False                False   
3                 False                 False                 True   
4                 False                 False                 True   

   APPLICATION_TYPE_T4  APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  ...  \
0                False                False                False  ...   
1                False                False                False  

In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df_encoded['IS_SUCCESSFUL']  # Target variable
X = application_df_encoded.drop(columns=['IS_SUCCESSFUL'])  # Features

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting datasets
print(f"Training features shape: {X_train.shape}")
print(f"Testing features shape: {X_test.shape}")
print(f"Training target shape: {y_train.shape}")
print(f"Testing target shape: {y_test.shape}")

Training features shape: (27439, 50)
Testing features shape: (6860, 50)
Training target shape: (27439,)
Testing target shape: (6860,)


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [44]:
# Create a Keras Sequential model and add more than one Dense hidden layer
nn_model = tf.keras.models.Sequential()

nn_model.add(tf.keras.layers.Dense(units=10, activation="relu", input_dim=50))

nn_model.add(tf.keras.layers.Dense(units=10, activation="relu"))

nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 10)                510       
                                                                 
 dense_19 (Dense)            (None, 10)                110       
                                                                 
 dense_20 (Dense)            (None, 1)                 11        
                                                                 
Total params: 631
Trainable params: 631
Non-trainable params: 0
_________________________________________________________________


In [45]:
# Convert to NumPy arrays if not already
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

# Ensure the data types are correct
X_train = X_train.astype(float)
X_test = X_test.astype(float)
y_train = y_train.astype(int)  
y_test = y_test.astype(int)   

# Verify shapes
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(27439, 50) (6860, 50) (27439,) (6860,)


In [46]:
history = nn.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_data=(X_test_scaled, y_test), verbose=1)

Epoch 1/100
858/858 [==============================] - 0s 417us/step - loss: 0.5377 - accuracy: 0.7370 - val_loss: 0.5568 - val_accuracy: 0.7270
Epoch 2/100
858/858 [==============================] - 0s 354us/step - loss: 0.5375 - accuracy: 0.7366 - val_loss: 0.5593 - val_accuracy: 0.7271
Epoch 3/100
858/858 [==============================] - 0s 351us/step - loss: 0.5378 - accuracy: 0.7364 - val_loss: 0.5583 - val_accuracy: 0.7283
Epoch 4/100
858/858 [==============================] - 0s 374us/step - loss: 0.5375 - accuracy: 0.7375 - val_loss: 0.5569 - val_accuracy: 0.7299
Epoch 5/100
858/858 [==============================] - 0s 371us/step - loss: 0.5374 - accuracy: 0.7354 - val_loss: 0.5603 - val_accuracy: 0.7252
Epoch 6/100
858/858 [==============================] - 0s 361us/step - loss: 0.5378 - accuracy: 0.7373 - val_loss: 0.5577 - val_accuracy: 0.7261
Epoch 7/100
858/858 [==============================] - 0s 355us/step - loss: 0.5374 - accuracy: 0.7377 - val_loss: 0.5567 - val_ac

In [47]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=1)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 [==============================] - 0s 248us/step - loss: 0.5586 - accuracy: 0.7280
Loss: 0.5586397051811218, Accuracy: 0.7279883623123169


In [40]:
# Export our model to HDF5 file
model_filename = 'AlphabetSoupCharity_Optimization.h5'
nn.save(model_filename)

print(f"Model saved to {model_filename}")

Model saved to AlphabetSoupCharity_Optimization.h5
